In [5]:
import sys
import numpy as np
from pathlib import Path
from datetime import datetime, timedelta

sys.path.append("/data/common/dataiku/config/projects/MASK2FORMER_1/lib/python/contrai/")
from transform.projection import Projector
from pyopensky.schema import StateVectorsData4
from traffic.data import opensky

ft = 0.3048
min_altitude = 15000  # feet
bounds = (-1, 46, 4.0, 51)
azimuth_zenith_grid = np.load("/data/dataiku/data/mask2former/azimuth_zenith_grid_IR_50KM_1026.npy")
projector = Projector(azimuth_zenith_grid)

# Set your target date here
target_date = datetime(2025, 4, 6)
start_time = target_date.replace(hour=12, minute=0, second=0)
end_time = target_date.replace(hour=16, minute=0, second=0)

out_file = Path(f"./{target_date.strftime('%Y-%m-%d')}_12-16.parquet")

print(f"Processing {start_time} to {end_time}...")
try:
    t = (
        opensky.history(
            start_time,
            end_time,
            bounds=bounds,
            cached=False,
        )
        .query(f"(altitude >= {min_altitude})")
        .assign_id()
        .resample("5s")
        .eval()
    )
    t = t.aircraft_data()
    df = t.data
    df.dropna(subset=["latitude", "longitude", "altitude"], inplace=True)
    res = projector.lonlat_to_azzen(df.longitude, df.latitude, df.geoaltitude * ft)
    df["azimuth"] = res[0]
    df["zenith"] = res[1]
    columns = [
        "flight_id", "timestamp", "icao24", "registration", "typecode", "callsign",
        "latitude", "longitude", "altitude", "geoaltitude", "groundspeed", "vertical_rate",
        "azimuth", "zenith",
    ]
    df[columns].dropna().to_parquet(out_file, index=False)
    print(f"Saved {out_file}")
except Exception as e:
    print(f"Error: {e}")

Processing 2025-04-06 12:00:00 to 2025-04-06 16:00:00...


RUNNING: : 77.3% [00:07, 9.82%/s]
DOWNLOAD: 850klines [02:09, 6.54klines/s]


Error: (trino.exceptions.TrinoQueryError) TrinoQueryError(type=INSUFFICIENT_RESOURCES, name=EXCEEDED_TIME_LIMIT, message="Query exceeded the maximum execution time limit of 30.00m", query_id=20260126_102458_26730_r4iwu)
(Background on this error at: https://sqlalche.me/e/20/dbapi)
